# Stroke Prediction Classification


### **Introduction:**

The 5th highest cause of death and the leading cause of disability in the United States is due in part to strokes (Cerebrovascular Accidents). Strokes are the result of an insufficient supply of oxygen and nutrients to the brain due to a blood vessel bursting. Subsequently, brain cells die, which can often cause right-sided weakness/paralysis, and sensory impairment.

**Research Question**:
Based on multiple numerical clinical features, is someone likely to experience a stroke (or have they already experienced one)?

This 12-column dataset pertaining to Medical Stroke Data was acquired from kaggle.com. It contains factors that may affect someone’s likelihood of experiencing a stroke, a record of whether or not the patient experienced a stroke, and identifying information.

In [1]:
library(repr)
library(tidyverse)
library(tidymodels)
library(RColorBrewer)
library(ggplot2)
library(themis)
options(repr.matrix.max.rows = 6)

set.seed(1234)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”


ERROR: Error: package or namespace load failed for ‘tidymodels’ in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 there is no package called ‘recipes’


### **Preliminary Exploratory Data Analysis:**

Although not all values are used, we must ensure that we properly tidy the data into a usable format. Tidy data entails:
* Each row has a single observation.
* Each column has a single variable.
* Each value is in a single cell.

The data includes all these components except for the fact that there are N/A values in the bmi column of the dataset. However, the missing values must be counted later on once the data is split into the training set. Consequently, the missing vales will remain in the dataset until after the missing values are counted.

Changing each variable to its correct type can be beneficial (even if it is not used), since it will help with usability later on if needed. This can be done before the data is split into the training and testing set. Patient ID can be de-selected since identifying information is not relevant to the project.

In [2]:
url <- "https://raw.githubusercontent.com/romansinkus/DS_Group_Project/main/healthcare-dataset-stroke-data.csv"
untidy_stroke_data <- read_csv(url)
stroke_data <- untidy_stroke_data %>%
    select(-id) %>% 
    mutate(bmi = as.numeric(bmi)) %>% 
    rename(residence_type = Residence_type) %>% 
    mutate(across(c(gender, hypertension:residence_type, smoking_status:stroke), as_factor))

stroke_data

Parsed with column specification:
cols(
  id = col_double(),
  gender = col_character(),
  age = col_double(),
  hypertension = col_double(),
  heart_disease = col_double(),
  ever_married = col_character(),
  work_type = col_character(),
  Residence_type = col_character(),
  avg_glucose_level = col_double(),
  bmi = col_character(),
  smoking_status = col_character(),
  stroke = col_double()
)

Warning message:
“Problem with `mutate()` input `bmi`.
ℹ NAs introduced by coercion
ℹ Input `bmi` is `as.numeric(bmi)`.”
Warning message in mask$eval_all_mutate(dots[[i]]):
“NAs introduced by coercion”


gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke
<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>
Male,67,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
Female,61,0,0,Yes,Self-employed,Rural,202.21,NA,never smoked,1
Male,80,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
Female,49,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
Female,79,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
Male,81,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
Male,74,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
Female,69,0,0,No,Private,Urban,94.39,22.8,never smoked,1
Female,59,0,0,Yes,Private,Rural,76.15,NA,Unknown,1


Now that the data has been optimized for usability (but not completely tidied), it can be split into testing and training data (75% will be allocated as training data).

In [3]:
stroke_split <- initial_split(stroke_data, prop = 0.75, strata = stroke)
initial_stroke_train <- training(stroke_split)
initial_stroke_test <- testing(stroke_split)

ERROR: Error in initial_split(stroke_data, prop = 0.75, strata = stroke): could not find function "initial_split"


Below is a table summarizing the three different numerical variables that are possible predictors for the model. The mean values have been calculated for each.

In [ ]:
avg_values <- initial_stroke_train %>% 
    summarize(age_avg = mean(age, na.rm = TRUE),
            avg_glucose_level_avg = mean(avg_glucose_level, na.rm = TRUE),
            bmi_avg = mean(bmi, na.rm = TRUE))
avg_values

Based on the summarization of the training data, most patients indicated they were middle-aged and had healthy average glucose levels (healthy range: 90-110 mg/dL). The average patient observed exhibited a body mass index indicating they were in the "overweight" category (25.0—29.9).

Kaggle (the website where the dataset was collected) indicated missing values for the bmi column of the dataset. Shown below is the number of rows that include missing values.

In [ ]:
missing_bmi <- initial_stroke_train %>%
    select(bmi) %>%
    count(is.na(bmi))
missing_bmi

The stroke variable is the target variable for this model. Below is a table containing the number of patients who indicated they had experienced a stroke (1 = yes, 0 = no).

In [ ]:
categorical_values <- initial_stroke_train %>%
    group_by(stroke) %>% 
    summarize(n = n())

categorical_values

Now that the missing values have been counted and displayed, they can be removed to make the data tidy (each value must be in a single cell).

In [ ]:
stroke_train <- initial_stroke_train %>% 
    filter(!is.na(bmi))

stroke_test <- initial_stroke_test %>% 
    filter(!is.na(bmi))

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 7)

age_vs_avg_glucose_level_plot <- stroke_train %>% 
    ggplot(aes(x = age, y = avg_glucose_level))+
    geom_point(aes(color = stroke, shape = stroke), alpha = 0.6)+
    labs(x = "Age (years)", y = "Average Glucose Level (mg/dL)", fill = "Stroke")+
    ggtitle("Age vs. Average Glucose Level")+
    theme(text = element_text(size = 20))+
    #scale_fill_brewer(palette = "Set2")
    scale_colour_manual(values = c("green", "black"))

age_vs_avg_glucose_level_plot

This plot shows two possible predictors that can be used to classify the target variable (stroke). The higher the age of a patient, the greater amount of patients appear to have had strokes. However, there does not seem to be a greater number patients with strokes as the average glucose level increases or decreases.

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 7)

age_vs_bmi_plot <- stroke_train %>% 
    ggplot(aes(x = age, y = bmi))+
    geom_point(aes(color = stroke, shape = stroke), alpha = 0.6)+
    labs(x = "Age (years)", y = "Body Mass Index", fill = "Stroke")+
    ggtitle("Age vs. BMI")+
    theme(text = element_text(size = 20))+
    scale_colour_manual(values = c("green", "black"))

age_vs_bmi_plot

This plot shows a greater density of stroke points in the upper, right hand corner. This means that patients with a higher BMI and older age were more prone to suffering from strokes.

In [ ]:
bmi_vs_avg_glucose_level_plot <- stroke_train %>% 
    ggplot(aes(y = bmi, x = avg_glucose_level))+
    geom_point(aes(color = stroke, shape = stroke), alpha = 0.6)+
    labs(y = "Body Mass Index", x = "Average Glucose Level", fill = "Stroke")+
    ggtitle("BMI vs. Average Glucose Level")+
    theme(text = element_text(size = 20))+
    scale_colour_manual(values = c("green", "black"))

bmi_vs_avg_glucose_level_plot

This plot shows a somewhat even distribution of the stroke points from left to right, however, there is much greater density of non-stroke points on the left side (lower average glucose level). This means that people with higher average glucose levels are more likely to experience strokes based on the graph. This relationship was unclear in the last graph containing average glucose level, likely due to it being graphed on the y-axis. The stroke points also correlate to a higher BMI.

### **Methods:**

For our data analysis, we created 3 plots to see the relationship between BMI, age and average glucose level of the patient to see which factors contributed to strokes. 3 numerical variables in the dataset were considered because numerical variables best contribute to the kknn classification model. After observing all plots, we concluded that we would use all three variables examined because each showed a correlation to strokes (higher BMI, higher average glucose level, and older age).

After creating a summary table analyzing the number of people that suffered from a stroke, we found a class imbalance as the 190 experienced a stroke whereas 3643 did not, which can lead to bias with the classifier. To solve this problem, when creating the classification model, we will rebalance the data by oversampling the stroke = 1 case to give it more voting power, resulting in a more accurate prediction.

To predict the testing set, we will use cross-validation to split our training data. Then, we will create a recipe that specifies the class label and our 3 predictors and create nearest_neighbor model specification to choose the k with the highest accuracy. Consequently, we add the model and recipe to a workflow and pick a suitable k value. We will then use this value to come up with a prediction subsequently calculate the accuracy of our model.

We will visualize the results by plotting the different predictors (using multiple different graphs), and whether or not the predictions were accurate.

### **Expected Outcomes and Significance:**

We expect to find a trend that can help determine if a patient is more likely to suffer from a stroke. For example, being overweight or having a higher bmi, would mean the patient is more likely to suffer from a stroke. Also, patients of older age would be of higher risk of having a stroke. 

Using this prediction, predicting a stroke would be an easier and more efficient process. It could help many patients seek help at an early stage. Using this classification model, doctors and patients can easily input different diagnostics and obtain a prediction of whether they are at-risk for having a stroke, based on data from past patients. This could be used as a preliminary step in the process to diagnosing patients (or patients seeking diagnosis).

This could lead to other questions such as what other numerical clinical features contribute to strokes, and can they improve the accuracy of the model? Additionally, this same method can be used to detemine predictions for other illnesses.

### **START OF PROJECT**

In [ ]:
stroke_data_scaled <- stroke_data %>% 
    mutate(scaled_age = scale(age, center = TRUE),
            scaled_bmi = scale(bmi, center = TRUE),
            scaled_avg_glucose_level = scale(avg_glucose_level, center = TRUE))

In [ ]:
ups_recipe <- recipe(stroke ~ age + bmi + avg_glucose_level, data = stroke_train) %>% 
    step_upsample(stroke, over_ratio = 1, skip = FALSE) %>% 
    prep()

upsampled_stroke <- bake(ups_recipe, stroke_data)

upsampled_stroke %>% 
    group_by(stroke) %>% 
    summarize(n = n())

In [ ]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
  set_engine("kknn") %>%
  set_mode("classification")
knn_spec